# Unity Catalog Governance - Demo

**Cel szkoleniowy:** Opanowanie Unity Catalog jako platformy governance dla Databricks Lakehouse, zarządzanie dostępami, data masking, lineage i audit logging

**Zakres tematyczny:**
- Unity Catalog Architecture: Metastore, Catalog, Schema, Tables/Views/Volumes
- Zarządzanie dostępami: GRANT/REVOKE privileges
- Data Masking i Row-Level Security
- Data Lineage i Audit Logging
- Delta Sharing - secure data sharing
- Best Practices for Data Governance

---

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 3 - Transformation, Governance & Integrations
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony (wymagane!)
  - Uprawnienia: CREATE CATALOG, CREATE SCHEMA, GRANT/REVOKE
  - Klaster: Standard z minimum 2 workers
- **Czas trwania**: 45 minut
- **Prerekvizity**: 03_databricks_jobs_orchestration.ipynb

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie Unity Catalog jako zunifikowanej platformy governance dla data lakehouse

**Podstawowe pojęcia:**
- **Unity Catalog**: Zunifikowane rozwiązanie governance dla wszystkich data assets
- **Metastore**: Region-level container dla katalogów (top-level)
- **Three-level namespace**: catalog.schema.table
- **Securable objects**: Tables, Views, Functions, Volumes, Models
- **Fine-grained access control**: Table, column, row-level security
- **Automatic lineage**: End-to-end data flow tracking bez instrumentacji

**Hierarchia obiektów Unity Catalog:**
```
Metastore (region-level)
    ↓
Catalog (domain/environment)
    ↓
Schema (namespace/layer)
    ↓
Securable Objects:
    - Tables / Views (data)
    - Functions (UDF, stored procedures)
    - Volumes (file storage)
    - Models (ML models)
```

**Kluczowe cechy:**
- **Unified governance**: Jedna platforma dla danych, ML, BI
- **ACID transactions**: Gwarancje transakcyjne na poziomie katalogu
- **Audit logging**: Who accessed what and when
- **Data discovery**: Metadata search i tagging
- **Delta Sharing**: Secure cross-organization sharing

**Dlaczego to ważne?**
Unity Catalog rozwiązuje fundamentalne problemy governance w data lake:
- Brak centralnej kontroli dostępu
- Trudności z śledzeniem lineage
- Brak audytu dostępu do danych
- Problemy z compliance (GDPR, HIPAA)
- Silosy danych między zespołami

Unity Catalog zapewnia enterprise-grade governance przy zachowaniu flexibility data lakehouse.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i wyświetlenie kontekstu użytkownika:

In [0]:
# Ścieżki do katalogów z danymi (podkatalogi w DATASET_BASE_PATH z 00_setup)
CUSTOMERS_PATH = f"{DATASET_BASE_PATH}/customers"
ORDERS_PATH = f"{DATASET_BASE_PATH}/orders"
PRODUCTS_PATH = f"{DATASET_BASE_PATH}/products"

# Ścieżki do konkretnych plików
CUSTOMERS_CSV = f"{CUSTOMERS_PATH}/customers.csv"
ORDERS_JSON = f"{ORDERS_PATH}/orders_batch.json"
PRODUCTS_PARQUET = f"{PRODUCTS_PATH}/products.parquet"

**Konfiguracja ścieżek do plików danych:**

- **Customers CSV**: `{DATASET_BASE_PATH}/customers/customers.csv`
- **Orders JSON**: `{DATASET_BASE_PATH}/orders/orders_batch.json`  
- **Products Parquet**: `{DATASET_BASE_PATH}/products/products.parquet`

Te ścieżki będą używane do wczytania danych do Unity Catalog.

## 2.1 Przygotowanie Danych z Dataset

Zanim przejdziemy do zarządzania dostępami, wczytamy rzeczywiste dane z katalogu dataset/, które będziemy używać w przykładach Unity Catalog.

In [0]:
customers_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(CUSTOMERS_CSV)

In [ ]:
customers_df.printSchema()

**Dane customers wczytane**

Wczytano dane klientów z pliku CSV. Schema został automatycznie wykryty przez `inferSchema=true`.

In [ ]:
display(customers_df.limit(5))

In [0]:
orders_df = spark.read.json(ORDERS_JSON)

In [ ]:
orders_df.printSchema()

**Dane orders wczytane**

Wczytano dane zamówień z pliku JSON. Spark automatycznie wykrył schema dla nested JSON structures.

In [ ]:
display(orders_df.limit(5))

In [0]:
products_df = spark.read.parquet(PRODUCTS_PARQUET)

In [ ]:
products_df.printSchema()

**Dane products wczytane**

Wczytano dane produktów z pliku Parquet. Parquet zawiera optimized binary format with schema embedded in the file.

In [ ]:
display(products_df.limit(5))

## 1️⃣ Unity Catalog Architecture

**Unity Catalog** to zunifikowane rozwiązanie governance dla Databricks Lakehouse.

### Hierarchia obiektów:

```
Metastore (region-level)
    ↓
Catalog (database/domain)
    ↓
Schema (namespace)
    ↓
Securable Objects:
    - Tables / Views
    - Functions (UDF, stored procedures)
    - Volumes (files storage)
    - Models (ML models)
```

### Three-level namespace:
```sql
catalog.schema.table
```

Przykład:
```sql
main.sales.orders
dev.analytics.customer_metrics
prod.gold.daily_revenue
```

### Kluczowe cechy:
- **Unified governance**: jedna platforma dla danych, ML, BI
- **Fine-grained access control**: table, column, row level
- **Automatic lineage**: end-to-end data flow tracking
- **Audit logging**: who accessed what and when
- **Data discovery**: metadata search i tagging

---

## 📋 Setup i Basic Operations

### Creating Catalogs and Schemas:

In [0]:
# Create Catalog
spark.sql(f"""
    CREATE CATALOG IF NOT EXISTS {CATALOG}
    COMMENT 'Katalog KION dla danych szkoleniowych'
""")

**Weryfikacja listy katalogów:**

In [ ]:
# List catalogs
spark.sql("SHOW CATALOGS").display()

**Katalog utworzony/zweryfikowany**

Katalog służy jako top-level container dla wszystkich schematów i tabel w naszym środowisku szkoleniowym.

In [0]:
# Create Schemas within catalog
spark.sql(f"""
  CREATE SCHEMA IF NOT EXISTS {CATALOG}.{BRONZE_SCHEMA}
  COMMENT 'Bronze layer - surowe dane'
""")

spark.sql(f"""
  CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SILVER_SCHEMA}
  COMMENT 'Silver layer - oczyszczone dane'
""")

spark.sql(f"""
  CREATE SCHEMA IF NOT EXISTS {CATALOG}.{GOLD_SCHEMA}
  COMMENT 'Gold layer - dane biznesowe'
""")

**Schematy Bronze, Silver, Gold utworzone**

Implementujemy architekturę Medallion:
- **Bronze**: Surowe dane (raw data ingestion)
- **Silver**: Oczyszczone dane (data validation, normalization)  
- **Gold**: Dane biznesowe (aggregated, business-ready)

In [0]:
# Ustawienie default catalog i schema
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SILVER_SCHEMA}")

**Weryfikacja aktywnego kontekstu:**

In [ ]:
# Verify the active catalog and schema
print(f"✓ Aktywny katalog: {CATALOG}")
print(f"✓ Aktywny schemat: {SILVER_SCHEMA}")

# Weryfikacja utworzonych schematów
schemas = spark.sql(f"SHOW SCHEMAS IN {CATALOG}").select("databaseName").collect()
schema_names = [row.databaseName for row in schemas]

print("\n✓ Utworzone schematy w katalogu:")
for schema_name in schema_names:
    print(f"  - {schema_name}")

**Aktywny katalog i schemat ustawione**

Ustawiamy domyślny kontekst pracy - wszystkie kolejne operacje będą wykonywane w tym katalogu i schemacie, chyba że zostanie podana pełna ścieżka.

In [ ]:
# Weryfikacja utworzonych schematów
spark.sql(f"SHOW SCHEMAS IN {CATALOG}").display()

### Creating Tables in Unity Catalog:

In [0]:
# Zapisanie tabeli customers w Bronze layer
customers_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.customers")

**Tabela customers zapisana w Bronze layer**

Zapisujemy dane w formacie Delta Lake z `mergeSchema=true` dla automatycznego dostosowania schematu w przyszłości.

In [ ]:
# Weryfikacja liczby rekordów
spark.sql(f"SELECT COUNT(*) as count FROM {CATALOG}.{BRONZE_SCHEMA}.customers").display()

In [0]:
# Zapisanie tabeli orders w Bronze layer
orders_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.orders")

print(f"✓ Tabela orders zapisana w {CATALOG}.{BRONZE_SCHEMA}")

**Weryfikacja tabeli orders:**

In [ ]:
# Weryfikacja
result = spark.sql(f"SELECT COUNT(*) as count FROM {CATALOG}.{BRONZE_SCHEMA}.orders").collect()[0]
print(f"✓ Liczba rekordów: {result.count}")

In [ ]:
# Add table properties and comments
spark.sql(f"""
    ALTER TABLE {CATALOG}.{BRONZE_SCHEMA}.orders
    SET TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'true',
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true',
        'owner' = 'data-engineering-team',
        'department' = 'analytics',
        'pii_data' = 'true'
    )
""")

In [ ]:
# Add comments to table and columns
spark.sql(f"""
    COMMENT ON TABLE {CATALOG}.{BRONZE_SCHEMA}.orders IS
    'Cleaned orders table with data quality validations applied'
""")

spark.sql(f"""
    COMMENT ON COLUMN {CATALOG}.{BRONZE_SCHEMA}.orders.customer_id IS
    'Customer identifier - PII data, access restricted'
""")

**Właściwości tabeli i komentarze dodane**

Ustawiliśmy:
- **Change Data Feed**: Śledzenie zmian w tabeli
- **Auto Optimize**: Automatyczna optymalizacja zapisu i kompakcja
- **Metadata**: Owner, department, oznaczenie PII data
- **Dokumentacja**: Komentarze do tabeli i wrażliwych kolumn

In [ ]:
# Weryfikacja liczby rekordów orders
spark.sql(f"SELECT COUNT(*) as count FROM {CATALOG}.{BRONZE_SCHEMA}.orders").display()

In [0]:
# Zapisanie tabeli products w Bronze layer
products_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.products")

print(f"✓ Tabela products zapisana w {CATALOG}.{BRONZE_SCHEMA}")

In [ ]:
# Weryfikacja liczby rekordów products
spark.sql(f"SELECT COUNT(*) as count FROM {CATALOG}.{BRONZE_SCHEMA}.products").display()

## 2.2 Data Classification (Tagging)

**Tagging** pozwala na klasyfikację danych (np. PII, Sensitive, GDPR) na poziomie tabeli lub kolumny.
Ułatwia to data discovery oraz governance (np. raportowanie wszystkich tabel zawierających dane osobowe).


In [ ]:
# Dodaj tagi do tabeli orders
spark.sql(f"""
    ALTER TABLE {CATALOG}.{BRONZE_SCHEMA}.orders 
    SET TAGS ('sensitivity' = 'high', 'domain' = 'sales')
""")

# Dodaj tagi do kolumny customer_id
spark.sql(f"""
    ALTER TABLE {CATALOG}.{BRONZE_SCHEMA}.orders 
    ALTER COLUMN customer_id SET TAGS ('pii' = 'true')
""")

print("✓ Tagi dodane do tabeli i kolumny")

**Wyszukiwanie po tagach (Information Schema):**

In [ ]:
# Znajdź wszystkie kolumny oznaczone jako PII
pii_columns = spark.sql(f"""
    SELECT 
        catalog_name, 
        schema_name, 
        table_name, 
        column_name, 
        tag_value 
    FROM system.information_schema.column_tags
    WHERE tag_name = 'pii' AND tag_value = 'true'
      AND catalog_name = '{CATALOG}'
""")

display(pii_columns)

**Tabela products zapisana w Bronze layer**

Wszystkie trzy tabele (customers, orders, products) zostały zapisane w Bronze schema w formacie Delta Lake. Gotowe do dalszej transformacji w architekturze Medallion.

## 4. Unity Catalog Volumes

**Volumes** to zarządzane przestrzenie dla przechowywania plików (non-tabular data) w Unity Catalog:
- **Managed Volumes**: Databricks zarządza cyklem życia plików
- **External Volumes**: połączenie z zewnętrznymi lokalizacjami storage

**Zastosowania**:
- Przechowywanie plików ML models, checkpoints
- Staging area dla danych przed ingestion
- Archiwum dokumentów, logów, raportów

In [0]:
# Tworzenie Managed Volume
volume_name = "files"

spark.sql(f"""
  CREATE VOLUME IF NOT EXISTS {CATALOG}.{BRONZE_SCHEMA}.{volume_name}
  COMMENT 'Managed volume dla plików staging'
""")

**Volume 'files' utworzony**

Volume został utworzony w Bronze schema jako managed volume dla przechowywania plików staging, dokumentów i innych zasobów niebędących tabelami.

In [0]:
# Eksport customers do CSV w Volume
volume_path = f"/Volumes/{CATALOG}/{BRONZE_SCHEMA}/{volume_name}"

customers_df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(f"{volume_path}/customers_export")

**Dane customers wyeksportowane do Volume**

Dane zostały zapisane w ścieżce: `/Volumes/{CATALOG}/{BRONZE_SCHEMA}/files/customers_export/`

Volume umożliwia przechowywanie plików niestrukturalnych obok tabeli w Unity Catalog z kontrolą dostępu.

In [0]:
# Weryfikacja plików w Volume
dbutils.fs.ls(f"{volume_path}/customers_export")

## 5. Unity Catalog Functions (UDF)

**Functions** w Unity Catalog pozwalają na:
- Tworzenie reużywalnych funkcji SQL/Python
- Centralne zarządzanie logiką biznesową
- Kontrolę dostępu przez GRANT/REVOKE
- Lineage tracking dla funkcji

**Rodzaje funkcji**:
- **Scalar Functions**: zwracają pojedynczą wartość
- **Table Functions**: zwracają tabelę
- **SQL Functions**: napisane w SQL
- **Python Functions**: napisane w Python (UDF)

In [0]:
# SQL Function - maskowanie customer_id
spark.sql(f"""
  CREATE OR REPLACE FUNCTION {CATALOG}.{SILVER_SCHEMA}.mask_customer_id(customer_id INT)
  RETURNS STRING
  LANGUAGE SQL
  COMMENT 'Maskuje customer_id, pokazując tylko ostatnie 3 cyfry'
  RETURN CONCAT('****', SUBSTRING(CAST(customer_id AS STRING), -3))
""")

**Funkcja mask_customer_id utworzona**

Funkcja SQL maskuje customer_id, pokazując tylko ostatnie 3 cyfry. Może być wykorzystana w Views do ukrywania wrażliwych identyfikatorów.

In [0]:
# Test funkcji mask_customer_id
result_df = spark.sql(f"""
  SELECT 
    customer_id,
    {CATALOG}.{SILVER_SCHEMA}.mask_customer_id(customer_id) as masked_id,
    first_name,
    last_name
  FROM {CATALOG}.{BRONZE_SCHEMA}.customers
  LIMIT 5
""")

display(result_df)

In [0]:
# Python UDF - kategoryzacja cen
spark.sql(f"""
  CREATE OR REPLACE FUNCTION {CATALOG}.{SILVER_SCHEMA}.categorize_price(price DOUBLE)
  RETURNS STRING
  LANGUAGE PYTHON
  COMMENT 'Kategoryzuje ceny: Low, Medium, High'
  AS $$
    if price < 50:
        return "Low"
    elif price < 200:
        return "Medium"
    else:
        return "High"
  $$
""")

**Funkcja categorize_price utworzona**

Funkcja Python UDF kategoryzuje ceny produktów:
- **Low**: < 50
- **Medium**: 50-200  
- **High**: > 200

Python UDF może zawierać dowolną logikę Python.

In [0]:
# Test funkcji categorize_price
result_df = spark.sql(f"""
  SELECT 
    product_name,
    price,
    {CATALOG}.{SILVER_SCHEMA}.categorize_price(price) as price_category
  FROM {CATALOG}.{BRONZE_SCHEMA}.products
  ORDER BY price
  LIMIT 10
""")

display(result_df)

In [0]:
# Tworzenie View w Silver layer - agregacja zamówień
spark.sql(f"""
  CREATE OR REPLACE VIEW {CATALOG}.{SILVER_SCHEMA}.customer_order_summary AS
  SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.country,
    COUNT(o.order_id) as total_orders,
    SUM(o.total_amount) as total_spent,
    AVG(o.total_amount) as avg_order_value,
    MAX(o.order_datetime) as last_order_date
  FROM {CATALOG}.{BRONZE_SCHEMA}.customers c
  LEFT JOIN {CATALOG}.{BRONZE_SCHEMA}.orders o
    ON c.customer_id = o.customer_id
  GROUP BY c.customer_id, c.first_name, c.last_name, c.country
""")

**Weryfikacja utworzenia View:**

In [ ]:
print(f"✓ View customer_order_summary utworzony w {CATALOG}.{SILVER_SCHEMA}")

**View customer_order_summary utworzony**

Silver layer View łączy customers i orders do agregacji per klient:
- **total_orders**: Liczba zamówień
- **total_spent**: Łączna wartość zamówień
- **avg_order_value**: Średnia wartość zamówienia
- **last_order_date**: Data ostatniego zamówienia

Gotowy do konsumowania przez Gold layer i BI tools.

---

## 6️⃣ Zarządzanie dostępami: GRANT / REVOKE

### Hierarchia Privileges w Unity Catalog:

**Poziomy uprawnień**:
1. **Metastore-level**: CREATE CATALOG, USE CATALOG
2. **Catalog-level**: USE CATALOG, CREATE SCHEMA
3. **Schema-level**: USE SCHEMA, CREATE TABLE, CREATE FUNCTION, CREATE VOLUME
4. **Object-level**: SELECT, MODIFY (INSERT/UPDATE/DELETE/MERGE), EXECUTE

**Securable Objects - Inheritance**:
- Uprawnienia dziedziczą się w dół hierarchii
- GRANT na Catalog → dziedziczy na wszystkie Schemas i Tables
- GRANT na Schema → dziedziczy na wszystkie Tables w tym Schema
- Można nadać uprawnienia na konkretnym poziomie dla fine-grained control

### Przykłady GRANT/REVOKE:

In [ ]:
# Setup: Create groups for demonstration purposes
# Note: This requires account admin privileges. If you don't have them, ensure these groups exist.
try:
    spark.sql("CREATE GROUP IF NOT EXISTS `data-analysts`")
    spark.sql("CREATE GROUP IF NOT EXISTS `data-engineers`")
    spark.sql("CREATE GROUP IF NOT EXISTS `finance-team`")
    spark.sql("CREATE GROUP IF NOT EXISTS `marketing-team`")
    print("✓ Groups created or already exist")
except Exception as e:
    print(f"⚠️ Could not create groups (likely due to permissions): {e}")
    print("Please ensure these groups exist in your Databricks Account or use existing groups.")

In [0]:
# Grant catalog access to data analysts
spark.sql(f"""
    GRANT USE CATALOG ON CATALOG {CATALOG} TO `data-analysts`
""")

spark.sql(f"""
    GRANT USE SCHEMA ON SCHEMA {CATALOG}.{SILVER_SCHEMA} TO `data-analysts`
""")

spark.sql(f"""
    GRANT SELECT ON SCHEMA {CATALOG}.{SILVER_SCHEMA} TO `data-analysts`
""")

**Uprawnienia dla data-analysts ustawione**

Grupa `data-analysts` otrzymała:
- **USE CATALOG**: Dostęp do katalogu
- **USE SCHEMA**: Dostęp do schematu Silver  
- **SELECT**: Odczyt danych ze schematu Silver

In [0]:
# Grant full access to data engineers
spark.sql(f"""
    GRANT USE CATALOG, CREATE SCHEMA ON CATALOG {CATALOG} TO `data-engineers`
""")

**Uprawnienia dla Data Analysts (Gold Layer):**

In [ ]:
# GRANT dla data-analysts na Gold schema
spark.sql(f"""
  GRANT USE SCHEMA ON SCHEMA {CATALOG}.{GOLD_SCHEMA} TO `data-analysts`
""")

spark.sql(f"""
  GRANT SELECT ON SCHEMA {CATALOG}.{GOLD_SCHEMA} TO `data-analysts`
""")

**Pełne uprawnienia dla Data Engineers:**

In [ ]:
# GRANT ALL PRIVILEGES dla data-engineers
spark.sql(f"""
  GRANT ALL PRIVILEGES ON SCHEMA {CATALOG}.{BRONZE_SCHEMA} TO `data-engineers`
""")

spark.sql(f"""
  GRANT ALL PRIVILEGES ON SCHEMA {CATALOG}.{SILVER_SCHEMA} TO `data-engineers`
""")

spark.sql(f"""
  GRANT ALL PRIVILEGES ON SCHEMA {CATALOG}.{GOLD_SCHEMA} TO `data-engineers`
""")

**Uprawnienia dla grup ustawione**

- **data-analysts**: USE SCHEMA + SELECT na Gold (tylko agregowane dane)
- **data-engineers**: ALL PRIVILEGES na Bronze/Silver/Gold (pełna kontrola nad data pipeline)

In [0]:
# GRANT na konkretnych tabelach
spark.sql(f"""
    GRANT SELECT ON TABLE {CATALOG}.{GOLD_SCHEMA}.customer_order_summary TO `finance-team`
""")

spark.sql(f"""
    GRANT SELECT ON TABLE {CATALOG}.{GOLD_SCHEMA}.customers_masked TO `marketing-team`
""")

**Weryfikacja nadania uprawnień:**

**Table-specific access control**

Fine-grained permissions:
- **finance-team**: Dostęp do customer_order_summary (revenue analysis)
- **marketing-team**: Dostęp do customers_masked (customer insights z maskowaniem PII)

In [0]:
# GRANT EXECUTE na Functions
spark.sql(f"""
  GRANT EXECUTE ON FUNCTION {CATALOG}.{SILVER_SCHEMA}.mask_customer_id TO `data-analysts`
""")

spark.sql(f"""
  GRANT EXECUTE ON FUNCTION {CATALOG}.{SILVER_SCHEMA}.categorize_price TO `data-analysts`
""")

**EXECUTE permissions na funkcjach**

Grupa `data-analysts` może wykonywać:
- **mask_customer_id**: Maskowanie identyfikatorów klientów
- **categorize_price**: Kategoryzacja cen produktów

Functions w Unity Catalog mają own access control layer.

In [0]:
# Weryfikacja uprawnień na tabeli
spark.sql(f"""
    SHOW GRANTS ON TABLE {CATALOG}.{BRONZE_SCHEMA}.customers
""").display()

**Uprawnienia na tabeli customers**

`SHOW GRANTS` pokazuje wszystkie uprawnienia na konkretnej tabeli:
- **Principal**: User/grupa z uprawnieniami
- **Action**: Typ uprawnienia (SELECT, MODIFY, etc.)
- **Object**: Scoped object (table, schema, catalog)

### Ownership and transfer:

---

## 3️⃣ Data Masking i Row-Level Security

### Column-level masking (Dynamic Views):

Użyj funkcji `current_user()` i `is_account_group_member()` do conditional masking:

In [0]:
# Create masked view for PII data
spark.sql(f"""
  CREATE OR REPLACE VIEW {CATALOG}.{GOLD_SCHEMA}.customers_masked AS
  SELECT 
    customer_id,
    CASE 
      WHEN is_account_group_member('pii-access-group') THEN first_name
      ELSE CONCAT(LEFT(first_name, 1), '***')
    END as first_name,
    CASE 
      WHEN is_account_group_member('pii-access-group') THEN last_name
      ELSE CONCAT(LEFT(last_name, 1), '***')
    END as last_name,
    city,
    country,
    registration_date
  FROM {CATALOG}.{BRONZE_SCHEMA}.customers
""")

**View customers_masked utworzony**

View z dynamicznym maskowaniem danych PII:
- **pii-access-group**: Widzi pełne imiona i nazwiska  
- **Inne grupy**: Widzi tylko pierwszą literę + `***`

Maskowanie oparte na `is_account_group_member()` jest dynamiczne - bez duplikowania danych.

In [0]:
# Test View z maskowaniem
result_df = spark.sql(f"""
  SELECT * FROM {CATALOG}.{GOLD_SCHEMA}.customers_masked LIMIT 10
""")

display(result_df)

**Dane z maskowaniem wyświetlone**

Imiona i nazwiska są zamaskowane dla użytkowników bez uprawnień `pii-access-group`. Widoczne są tylko pierwsze litery + `***`.

In [0]:
# Tworzenie view z zahashowanym customer_id
spark.sql(f"""
  CREATE OR REPLACE VIEW {CATALOG}.{GOLD_SCHEMA}.orders_hashed AS
  SELECT 
    order_id,
    SHA2(CAST(customer_id AS STRING), 256) as customer_id_hash,
    product_id,
    quantity,
    total_amount,
    order_datetime,
    status
  FROM {CATALOG}.{BRONZE_SCHEMA}.orders
""")

print(f"✓ View orders_hashed utworzony - customer_id jest zahashowany")
print("  - Analitycy mogą agregować bez ujawniania customer_id")

**View orders_hashed utworzony**

Customer_id jest zahashowany przy użyciu SHA2-256. Umożliwia to:
- **Analitykom**: Agregację danych bez ujawniania customer_id
- **Privacy**: Zachowanie anonimowości przy zachowaniu możliwości grupowania
- **Compliance**: Spełnienie wymagań GDPR/privacy regulations

### Row-Level Security (RLS):

Restrict which rows users can see based on their identity or group membership:

In [0]:
# Tworzenie RLS view - dostęp per kraj
spark.sql(f"""
    CREATE OR REPLACE VIEW {CATALOG}.{GOLD_SCHEMA}.customers_rls AS
    SELECT *
    FROM {CATALOG}.{BRONZE_SCHEMA}.customers
    WHERE 
        CASE 
            WHEN is_account_group_member('global-access') THEN TRUE
            WHEN is_account_group_member('poland-team') THEN country = 'Poland'
            WHEN is_account_group_member('germany-team') THEN country = 'Germany'
            WHEN is_account_group_member('france-team') THEN country = 'France'
            ELSE FALSE
        END
""")

print(f"✓ RLS View utworzony - użytkownicy widzą tylko klientów ze swojego kraju")

**RLS View customers_rls utworzony**

Row-Level Security filtruje dane bazując na group membership:
- **global-access**: Widzi wszystkich klientów
- **poland-team**: Tylko klientów z Polski  
- **germany-team**: Tylko klientów z Niemiec
- **france-team**: Tylko klientów z Francji
- **Inne grupy**: Brak dostępu (FALSE)

Automatyczne filtrowanie wierszy bez duplikowania danych.

In [0]:
# RLS na zamówieniach - filtrowanie per rola
spark.sql(f"""
  CREATE OR REPLACE VIEW {CATALOG}.{GOLD_SCHEMA}.orders_rls AS
  SELECT 
    o.*
  FROM {CATALOG}.{BRONZE_SCHEMA}.orders o
  WHERE 
    is_account_group_member('admin') OR
    (is_account_group_member('finance-team') AND o.status IN ('completed', 'shipped')) OR
    (is_account_group_member('warehouse-team') AND o.status IN ('pending', 'processing', 'shipped'))
""")

**Nadanie uprawnień do RLS Views:**

**RLS View dla orders utworzony**

Role-based filtering zamówień:
- **admin**: Widzi wszystkie zamówienia
- **finance-team**: Tylko completed i shipped (revenue-relevant)
- **warehouse-team**: pending, processing, shipped (operational orders)

Różne grupy widzą różne subsets danych z tej samej tabeli.

In [0]:
# GRANT dostęp do customers_rls
spark.sql(f"""
  GRANT SELECT ON VIEW {CATALOG}.{GOLD_SCHEMA}.customers_rls TO `all-users`
""")

**Nadanie uprawnień do orders_rls**

In [ ]:
# GRANT dostęp do orders_rls
spark.sql(f"""
  GRANT SELECT ON VIEW {CATALOG}.{GOLD_SCHEMA}.orders_rls TO `all-users`
""")

**Odebranie dostępu do tabel bazowych (Enforcement):**

In [ ]:
# Revoke direct access to base table
spark.sql(f"""
    REVOKE SELECT ON TABLE {CATALOG}.{BRONZE_SCHEMA}.orders FROM `all-users`
""")

**RLS Views - Access control setup**

Security pattern:
1. **GRANT SELECT** na RLS Views dla `all-users`
2. **REVOKE SELECT** na base tables (force użycia Views)
3. **Automatic filtering** bazowane na group membership

Users mogą SELECT z Views, ale nie z base tables - enforcement RLS.

---

## 4️⃣ Data Lineage i Audit Logging

### Querying Data Lineage:

Unity Catalog automatically tracks lineage for:
- Table → Table (ETL transformations)
- Notebook → Table (data writes)
- Dashboard → Table (BI queries)
- ML Model → Table (training data)

**General Table Lineage**

In [0]:
# Query table lineage z system tables
lineage_df = spark.sql(f"""
  SELECT 
    source_table_full_name,
    source_type,
    target_table_full_name,
    target_type,
    created_at,
    created_by
  FROM system.access.table_lineage
  WHERE target_table_full_name LIKE '{CATALOG}.%'
  ORDER BY created_at DESC
  LIMIT 50
""")

display(lineage_df)

**Lineage dla tabel w katalogu wyświetlony**

System automatycznie śledzi lineage dla:
- **Table → Table**: ETL transformations
- **Notebook → Table**: Data writes  
- **Dashboard → Table**: BI queries
- **ML Model → Table**: Training data

Lineage jest dostępny przez `system.access.table_lineage` bez dodatkowej instrumentacji.

**1. Upstream Lineage (Sources)**

In [0]:
# Find upstream dependencies (sources) for a table
upstream_df = spark.sql(f"""
    SELECT DISTINCT
        source_table_full_name,
        source_type
    FROM system.access.table_lineage
    WHERE target_table_full_name = '{CATALOG}.{SILVER_SCHEMA}.customer_order_summary'
""")

display(upstream_df)

**⬆️ Upstream: Tabele źródłowe dla customer_order_summary**

Pokazuje wszystkie tabele, które są używane jako źródła danych w View `customer_order_summary`. Pomoce w analizie impact analysis przy zmianach w upstream tables.

**2. Downstream Lineage (Consumers)**

In [0]:
# Find downstream dependencies (consumers) of a table
downstream_df = spark.sql(f"""
    SELECT DISTINCT
        target_table_full_name,
        target_type
    FROM system.access.table_lineage
    WHERE source_table_full_name = '{CATALOG}.{BRONZE_SCHEMA}.customers'
""")

display(downstream_df)

**⬇️ Downstream: Views/Tables korzystające z customers**

Pokazuje wszystkie Views i tabele, które konsumują dane z tabeli `customers`. Krytyczne dla understanding impact of changes i data governance.

**3. Column-Level Lineage**

In [0]:
# Column-level lineage (jeśli dostępny)
column_lineage = spark.sql(f"""
    SELECT 
        source_table_full_name,
        source_column_name,
        target_table_full_name,
        target_column_name,
        created_at
    FROM system.access.column_lineage
    WHERE target_table_full_name = '{CATALOG}.{SILVER_SCHEMA}.customer_order_summary'
    ORDER BY target_column_name
""")

display(column_lineage)

**📊 Column-level lineage dla customer_order_summary**

Unity Catalog śledzi lineage na poziomie kolumn - które kolumny w source tables wpływają na które kolumny w target table. Szczegółowa informacja dla data governance i impact analysis.

### Audit Logging:

Unity Catalog logs all access and operations:

**1. General Audit Logs**

In [0]:
# Query audit logs
audit_df = spark.sql("""
    SELECT 
        event_time,
        user_identity.email as user_email,
        service_name,
        action_name,
        request_params.full_name_arg as table_name,
        response.status_code,
        request_id
    FROM system.access.audit
    WHERE action_name IN ('getTable', 'createTable', 'deleteTable', 'updateTable')
        AND event_date >= current_date() - INTERVAL 7 DAYS
    ORDER BY event_time DESC
    LIMIT 100
""")
audit_df.display()

**2. Sensitive Data Access**

In [0]:
# Track who accessed sensitive tables
sensitive_access = spark.sql(f"""
    SELECT 
        event_time,
        user_identity.email as user,
        action_name,
        request_params.full_name_arg as table_accessed,
        source_ip_address
    FROM system.access.audit
    WHERE request_params.full_name_arg LIKE '{CATALOG}.%.customers%'
        AND action_name = 'getTable'
        AND event_date >= current_date() - INTERVAL 7 DAYS
    ORDER BY event_time DESC
    LIMIT 100
""")

display(sensitive_access)

**🔒 Audit logs: Dostęp do tabeli customers (ostatnie 7 dni)**

Monitoring dostępu do wrażliwych tabel z danymi PII:
- **Kto**: User email
- **Kiedy**: Event time  
- **Co**: Table name
- **Skąd**: Source IP address

Kluczowe dla compliance (GDPR, HIPAA) i security monitoring.

**3. Privilege Changes**

In [0]:
# Grant/Revoke audit trail
grant_audit = spark.sql("""
    SELECT 
        event_time,
        user_identity.email as admin_user,
        action_name,
        request_params.privilege as privilege_granted,
        request_params.securable_full_name as object_name,
        request_params.principal as grantee
    FROM system.access.audit
    WHERE action_name IN ('grantPrivilege', 'revokePrivilege')
        AND event_date >= current_date() - INTERVAL 30 DAYS
    ORDER BY event_time DESC
""")

display(grant_audit)

**📝 Audit trail of privilege changes**

Kompletny audit trail zmian uprawnień:
- **Admin user**: Kto wykonał GRANT/REVOKE
- **Action**: grantPrivilege lub revokePrivilege
- **Privilege**: Które uprawnienie (SELECT, MODIFY, etc.)
- **Object**: Na którym obiekcie (table, schema, catalog)
- **Grantee**: Komu nadano/odebrano uprawnienia

Niezbędne dla governance i compliance audits.

---

## 5️⃣ Delta Sharing

**Delta Sharing** = Secure data sharing protocol (cross-org, cross-cloud)

### Komponenty:
- **Share**: kolekcja tabel do udostępnienia
- **Recipient**: organizacja/użytkownik otrzymujący dane
- **Provider**: właściciel danych (Ty)

### Create Share:

In [0]:
# Tworzenie Share dla zewnętrznych partnerów
share_name = f"{CATALOG}_partner_share"

spark.sql(f"""
  CREATE SHARE IF NOT EXISTS {share_name}
  COMMENT 'Udostępnienie danych KION dla partnerów biznesowych'
""")

**Share '{share_name}' utworzony**

Delta Sharing Share to kolekcja tabel do bezpiecznego udostępnienia zewnętrznym partnerom:
- **Cross-org**: Między różnymi organizacjami Databricks
- **Cross-cloud**: AWS ↔ Azure ↔ GCP  
- **Open protocol**: Standard open-source

In [0]:
# Dodanie tabeli do Share (tylko Gold layer - agregowane dane)
spark.sql(f"""
  ALTER SHARE {share_name}
  ADD TABLE {CATALOG}.{GOLD_SCHEMA}.customer_order_summary
""")

**Tabela customer_order_summary dodana do Share**

Best practice: Udostępniaj tylko Gold layer (agregowane dane):
- **Bezpieczeństwo**: Brak dostępu do raw data
- **Privacy**: Agregacje ukrywają individual records
- **Stability**: Gold layer ma stabilny schema i strukture

In [0]:
# Weryfikacja zawartości Share
spark.sql(f"SHOW ALL IN SHARE {share_name}").display()

**Tabele w Share zweryfikowane**

Share zawiera obecnie dodane tabele i może być udostępniony recipientom. Recipients otrzymają activation link do konsumowania shared data przez Delta Sharing protocol.

### Create Recipient:

### Consuming shared data (as recipient):

### Best practices for Delta Sharing:

1. **Share only aggregated/gold data**: nie udostępniaj raw/bronze layers
2. **Use views for masking**: create view with masked PII before sharing
3. **Monitor access**: track who accesses shared data
4. **Version control**: use table versions for stable APIs
5. **Documentation**: clear documentation dla recipients

---

## 6️⃣ Best Practices for Data Governance

### 1. Catalog organization strategy:

### 2. Access control patterns:

### 3. Tagging and documentation:

### 4. Monitoring and alerts:

In [ ]:
# 1. Tabele bez właścicieli
unowned_tables = spark.sql(f"""
    SELECT 
        table_catalog,
        table_schema,
        table_name
    FROM system.information_schema.tables
    WHERE table_catalog = '{CATALOG}'
        AND table_owner IS NULL
""")

display(unowned_tables)

**⚠️ Tabele bez właścicieli**

Każda tabela powinna mieć przypisanego owner dla accountability i governance. Tabele bez owner są problematyczne dla zarządzania dostępami i lifecycle management.

In [ ]:
# 2. Tabele bez dokumentacji
undocumented = spark.sql(f"""
    SELECT 
        table_catalog,
        table_schema,
        table_name
    FROM system.information_schema.tables
    WHERE table_catalog = '{CATALOG}'
        AND (comment IS NULL OR comment = '')
""")

display(undocumented)

**📝 Tabele bez dokumentacji**

Każda tabela powinna mieć `COMMENT` opisujący:
- **Cel tabeli**: Co zawiera i do czego służy
- **Source**: Skąd pochodzą dane
- **Owner**: Kto jest odpowiedzialny
- **Retention**: Jak długo dane są przechowywane

In [ ]:
# 3. Tabele nieużywane (brak queries w ostatnich 90 dniach)
unused_tables = spark.sql(f"""
    WITH recent_access AS (
        SELECT DISTINCT request_params.full_name_arg as table_name
        FROM system.access.audit
        WHERE action_name = 'getTable'
            AND event_date >= current_date() - INTERVAL 90 DAYS
    )
    SELECT 
        t.table_catalog,
        t.table_schema,
        t.table_name,
        t.created as table_created_at
    FROM system.information_schema.tables t
    LEFT JOIN recent_access ra 
        ON CONCAT(t.table_catalog, '.', t.table_schema, '.', t.table_name) = ra.table_name
    WHERE t.table_catalog = '{CATALOG}'
        AND ra.table_name IS NULL
        AND t.created < current_date() - INTERVAL 90 DAYS
""")

display(unused_tables)

**🗑️ Tabele nieużywane (90+ dni bez dostępu)**

Tabele bez dostępu w ostatnich 90 dniach mogą być kandydatami do:
- **Archiwizacji**: Przeniesienie do cold storage
- **Deprecation**: Oznaczenie jako deprecated  
- **Cleanup**: Usunięcie po weryfikacji z business stakeholders

Monitorowanie usage pomaga optimalizować koszty storage.

---

## ✅ Podsumowanie

### Nauczyłeś się:

✅ **Unity Catalog Architecture**: Metastore → Catalog → Schema → Tables  
✅ **Access Control**: GRANT/REVOKE privileges at multiple levels  
✅ **Data Masking**: Column-level masking with dynamic views  
✅ **Row-Level Security**: Filter data based on user identity  
✅ **Data Lineage**: Track data flow through system tables  
✅ **Audit Logging**: Monitor who accessed what and when  
✅ **Delta Sharing**: Secure cross-organization data sharing  

### Key Takeaways:

1. **Unified Governance**: Single platform for all data assets
2. **Fine-grained Control**: Table, column, row-level security
3. **Automatic Lineage**: No extra instrumentation needed
4. **Compliance-ready**: Audit logs for regulatory requirements
5. **Secure Sharing**: Delta Sharing for external collaboration

### Następne kroki:
- **Notebook 05**: BI & ML Integrations
- **Workshop 03**: Governance + Integrations hands-on

---

## 📚 Dodatkowe zasoby

- [Unity Catalog Documentation](https://docs.databricks.com/data-governance/unity-catalog/index.html)
- [Delta Sharing Protocol](https://delta.io/sharing/)
- [Unity Catalog Best Practices](https://docs.databricks.com/data-governance/unity-catalog/best-practices.html)

---

## ✅ Checklist - Unity Catalog Governance

Po ukończeniu tego notebooka powinieneś umieć:

- [ ] **UC Architecture**: Zrozumieć hierarchię Metastore → Catalog → Schema → Objects
- [ ] **Tworzenie obiektów**: Utworzyć Catalog, Schema, Tables, Views, Volumes, Functions
- [ ] **GRANT/REVOKE**: Zarządzać uprawnieniami na wszystkich poziomach
- [ ] **Privileges**: Rozumieć SELECT, MODIFY, CREATE TABLE, EXECUTE
- [ ] **Data Masking**: Tworzyć Views z maskowaniem wrażliwych danych
- [ ] **Row-Level Security**: Implementować RLS bazowane na group membership
- [ ] **Lineage**: Śledzić upstream/downstream dependencies
- [ ] **Audit Logging**: Zapytywać system.access.audit o aktywność użytkowników
- [ ] **Delta Sharing**: Tworzyć Share i udostępniać dane zewnętrznym recipientom
- [ ] **Best Practices**: Monitorować governance health (owners, documentation, unused tables)

---

## 🔧 Troubleshooting

### Problem 1: "Table or view not found"
**Przyczyna**: Brak uprawnień USE CATALOG lub USE SCHEMA  
**Rozwiązanie**:
```sql
GRANT USE CATALOG ON CATALOG <catalog_name> TO <principal>;
GRANT USE SCHEMA ON SCHEMA <catalog>.<schema> TO <principal>;
```

### Problem 2: "Permission denied" przy SELECT
**Przyczyna**: Brak uprawnień SELECT na tabeli  
**Rozwiązanie**:
```sql
GRANT SELECT ON TABLE <catalog>.<schema>.<table> TO <principal>;
-- lub na całym schema:
GRANT SELECT ON SCHEMA <catalog>.<schema> TO <principal>;
```

### Problem 3: "Cannot execute function"
**Przyczyna**: Brak uprawnienia EXECUTE na funkcji  
**Rozwiązanie**:
```sql
GRANT EXECUTE ON FUNCTION <catalog>.<schema>.<function_name> TO <principal>;
```

### Problem 4: "Volume not accessible"
**Przyczyna**: Brak uprawnień READ VOLUME / WRITE VOLUME  
**Rozwiązanie**:
```sql
GRANT READ VOLUME ON VOLUME <catalog>.<schema>.<volume> TO <principal>;
GRANT WRITE VOLUME ON VOLUME <catalog>.<schema>.<volume> TO <principal>;
```

### Problem 5: RLS View nie filtruje danych
**Przyczyna**: Użytkownik nie należy do żadnej grupy zdefiniowanej w CASE WHEN  
**Rozwiązanie**: Dodaj użytkownika do odpowiedniej grupy lub dodaj domyślny fallback w View

### Problem 6: Lineage nie pokazuje zależności
**Przyczyna**: Lineage jest automatyczne, ale może opóźniać się o kilka minut  
**Rozwiązanie**: Poczekaj 5-10 minut i ponownie zapytaj system.access.table_lineage

### Problem 7: Share nie widoczny dla recipient
**Przyczyna**: Recipient nie aktywował activation link  
**Rozwiązanie**: Wyślij activation link z DESCRIBE RECIPIENT

---

## 🏆 Best Practices Summary

### 1. **Catalog Organization**
- ✅ Używaj environment-based catalogs: `dev`, `test`, `prod`
- ✅ Organizuj schematy według warstw: `bronze`, `silver`, `gold`
- ✅ Stosuj naming conventions: `<catalog>.<schema>.<object>`

### 2. **Access Control**
- ✅ **Principle of Least Privilege**: Nadawaj minimalne wymagane uprawnienia
- ✅ Używaj grup, nie indywidualnych użytkowników
- ✅ Inheritance: GRANT na Catalog → dziedziczy na Schema → dziedziczy na Tables
- ✅ Regularnie audytuj uprawnienia (SHOW GRANTS)

### 3. **Data Masking & RLS**
- ✅ Maskuj PII w Views dla użytkowników bez pii-access-group
- ✅ Używaj RLS dla multi-tenant scenarios
- ✅ Zawsze testuj masking z różnymi group membership

### 4. **Lineage & Audit**
- ✅ Wykorzystuj automatic lineage do śledzenia data flow
- ✅ Regularnie sprawdzaj audit logs dla sensitive tables
- ✅ Monitoruj lineage po zmianach w pipeline

### 5. **Delta Sharing**
- ✅ Udostępniaj tylko Gold layer (aggregated data)
- ✅ Używaj masked Views w Share
- ✅ Dokumentuj Share contracts dla recipients

### 6. **Documentation & Governance**
- ✅ Dodawaj COMMENT do wszystkich tabel, views, functions
- ✅ Używaj Table Properties dla metadata (owner, PII, retention)
- ✅ Regularnie sprawdzaj governance health checks

### 7. **Volumes & Functions**
- ✅ Używaj Managed Volumes dla ML artifacts i staging
- ✅ Centralizuj logikę biznesową w UC Functions
- ✅ Kontroluj dostęp przez GRANT EXECUTE

---